## Import and configure

In [ ]:
# Imports from standard library
import sys
import os
import datetime
import warnings
warnings.filterwarnings("ignore")

# Import of third party packages
import numpy as np
%matplotlib widget
from matplotlib import pyplot
import cmocean

# EAT itself
import eatpy

# Shared postprocessing scripts
sys.path.append("..")
import shared

# Experiment configuration
N = 20   # ensemble size
plot_start = datetime.datetime(2021,1,1)

# For automated testing we reduce the ensemble size
if "GITHUB_ACTIONS" in os.environ:
    N = 2

## Load observations

In [ ]:
sst_obs = shared.read_0d_observations("cci_sst.dat")
chl_obs = shared.read_0d_observations("cci_chl.dat")

# Undo log10 transformation for chlorophyll median and 25th and 75th percentiles
chl_obs[:, 1:] = 10.0 ** chl_obs[:, 1:]

# Reference simulation

Forecast-only, no data assimilation

In [ ]:
!eat-gotm

## Load results

In [ ]:
time, z, temp, _, _ = shared.read_result('result.nc', "temp")
time, z, chl, _, _ = shared.read_result('result.nc', "total_chlorophyll_calculator_result")
sst = temp[..., -1]
chl_sf = chl[..., -1]

## Plot temperature

In [ ]:
fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(figsize=(8,6), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex='col')

# Plot modelled and observed sea surface temperature
shared.plot_0d_timeseries(ax1, time, sst, sst_obs)
ax1.set_ylabel('temperature (°C)')
ax1.set_title('sea surface temperature')
cax1.axis('off')

# Plot modelled temperature throughout the water column
pc, cb = shared.plot_1d_timeseries(ax2, time, z, temp, cmap=cmocean.cm.thermal, cax=cax2)
cb.set_label('temperature (°C)')
ax2.set_title('model temperature')

fig.tight_layout()

fig.savefig('reference_sst.png', dpi=150)

## Plot chlorophyll

In [ ]:
fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(figsize=(8,6), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex='col')

shared.plot_0d_timeseries(ax1, time, chl_sf, chl_obs)
ax1.set_ylabel('chlorophyll (mg m-3)')
ax1.set_title('surface chlorophyll')
cax1.axis('off')

pc, cb = shared.plot_1d_timeseries(ax2, time, z, chl, cmap=cmocean.cm.algae, cax=cax2)
cb.set_label('chlorophyll (mg m-3)')
ax2.set_title('chlorophyll')

fig.tight_layout()
fig.savefig('reference_sst_chl.png', dpi=150)

# Physics-only data assimilation

## Create the ensemble

In [ ]:
# Vary wind speeds (x and y components) and background mixing (minimum turbulent kinetic energy) 
gotm = eatpy.models.gotm.YAMLEnsemble("gotm.yaml", N)
with gotm:
    gotm["surface/u10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["surface/v10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["turbulence/turb_param/k_min"] *= np.random.lognormal(sigma=0.2, size=N)

## Run data assimilation experiment

In [ ]:
!mpiexec -n 1 python assimilate_sst.py : -n {N} eat-gotm --separate_gotm_yaml

## Load results

In [ ]:
enstime, ensz, temps, _, _ = shared.read_ensemble_result('result.nc', 'temp', N)
enstime, ensz, chls, _, _ = shared.read_ensemble_result('result.nc', 'total_chlorophyll_calculator_result', N)
ssts = temps[:, :, -1]
chl_sf_phys_DA = chls[:, :, -1]

## Plot

In [ ]:
# Plot ensemble median sea surface temperature, along with original (no DA) result and observations
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(figsize=(8,8), nrows=3, ncols=2, width_ratios=[0.95, 0.05], sharex='col')

shared.plot_0d_ensemble_timeseries(ax1, enstime, ssts, [("model, no DA", sst)], sst_obs, plot_spread=False, label="model, DA")
ax1.set_xlim(plot_start or time[0], time[-1])
ax1.set_ylabel('temperature (°C)')
ax1.legend(loc=(0.25, 0.65))
ax1.set_title('sea surface temperature')
cax1.axis('off')

pc, cb = shared.plot_1d_ensemble_timeseries(ax2, enstime, ensz, temps, 10, cmap=cmocean.cm.thermal, cax=cax2)
cb.set_label('temperature (°C)')
ax2.set_title('temperature with assimilation of remotely sensed temperature');

temp_diff = temps - temp[-enstime.size:, :]
contours = np.linspace(-2.0, 2.0, 21)
pc, cb = shared.plot_1d_ensemble_timeseries(ax3, enstime, ensz, temp_diff, contours, cmap='RdBu_r', extend='both', cax=cax3)
cb.set_label('temperature difference (°C)')
ax3.set_title('impact of data assimilation on temperature (DA - no DA)');

ax1.set_xlim(plot_start or time[0], time[-1])

fig.tight_layout()

fig.savefig('ensemble_sst.png', dpi=150)

# Physics + biogeochemistry data assimilation

## Create the ensemble

In [ ]:
gotm = eatpy.models.gotm.YAMLEnsemble("gotm.yaml", N)
fabm = eatpy.models.gotm.YAMLEnsemble("fabm.yaml", N)
with gotm, fabm:
    gotm["surface/u10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["surface/v10/scale_factor"] = np.random.lognormal(sigma=0.2, size=N)
    gotm["turbulence/turb_param/k_min"] = 5e-6 * np.random.lognormal(sigma=0.3, size=N)
    gotm["fabm/yaml_file"] = fabm.file_paths
    fabm["instances/phy/parameters/mumax0"] *= np.random.lognormal(sigma=0.3, size=N)
    fabm["instances/dia/parameters/mumax0"] *= np.random.lognormal(sigma=0.3, size=N)

## Run data assimilation experiment

In [ ]:
!mpiexec -n 1 python assimilate_sst_chl.py : -n {N} eat-gotm --separate_gotm_yaml

## Load results

In [ ]:
enstime, ensz, ens, _, _ = shared.read_ensemble_result("result.nc", "total_chlorophyll_calculator_result", N)

## Plot

In [ ]:
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(
    figsize=(10, 10),
    nrows=3,
    ncols=2,
    sharex="col",
    width_ratios=[0.95, 0.05],
    height_ratios=[0.4, 0.3, 0.3],
)

# Surface chlorophyll
shared.plot_0d_ensemble_timeseries(
    ax1,
    enstime,
    ens[:, :, -1],
    [("model, no DA", chl_sf), ("model, phys DA", np.median(chl_sf_phys_DA, axis=0))],
    obs=chl_obs,
    plot_spread=False,
    label="model, phys+bgc DA",
)
ax1.set_ylabel(f"chlorophyll (mg m-3)")
ax1.set_title("surface chlorophyll")
ax1.set_ylim(0.0, 4.0)
cax1.axis("off")

# Original modelled chlorophyll throughout the water column [free run]
chl_contours = np.linspace(0.0, 5.0, 11)
cf, cb = shared.plot_1d_timeseries(
    ax2, time, z, chl, chl_contours, cmap=cmocean.cm.algae, extend="max", cax=cax2
)
cb.set_label("chlorophyll (mg m-3)")
ax2.set_title(f"chlorophyll without data assimilation")

# Modelled chlorophyll throughout the water column with data assimilaiton
cf, cb = shared.plot_1d_ensemble_timeseries(
    ax3, enstime, ensz, ens, chl_contours, cmap=cmocean.cm.algae, extend="max", cax=cax3
)
cb.set_label("chlorophyll (mg m-3)")
ax3.set_title(
    f"chlorophyll with assimilation of remotely sensed temperature and chlorophyll"
)

ax1.set_xlim(plot_start or time[0], time[-1])

fig.tight_layout()

fig.savefig("ensemble_sst_chl.png", dpi=150)